In [1]:
!pip install PyPDF2
!pip install pytesseract pdf2image pillow
!sudo apt-get install tesseract-ocr
!sudo apt-get install tesseract-ocr-rus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00a 0:00:01
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-rus
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 1274 kB of archives.
After this operation, 3877 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-rus all 1:4.00~git30-7274cfa-1 [1274 kB]
Fetched 1274 kB in 1s (2144 kB/s)          
Selecting previously unselected package tesseract-ocr-rus.
(Reading database ... 116071 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-rus_1%3a4.00~git30-7274cfa-1_all.

In [2]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 2564 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.4 [916 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.4 [174 kB]
Fetched 2564 kB in 2s (1700 kB/s)       
Selecting previously unselected package

In [3]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 17.5 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image
import PyPDF2
from tqdm.notebook import tqdm
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, Colors
from pdf2image import convert_from_path
import pytesseract

line_width = 2
font_size = 8
model_path = '/kaggle/input/yolo-10b-doclaynet/yolov10b-doclaynet.pt'  # Ensure you provide the correct path to the model
yolo_model = YOLO(model_path)

pdf_directory = "/kaggle/input/hack-pdfs/"
pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]

for file_name in tqdm(pdf_files):
    text_output_dir = f"/kaggle/working/{file_name}/text_from_pdf/"
    unstructured_output_dir = f"/kaggle/working/{file_name}/unstructured_data/"

    os.makedirs(text_output_dir, exist_ok=True)
    os.makedirs(unstructured_output_dir, exist_ok=True)
    
    pdf_document = os.path.join(pdf_directory, file_name)
    
    with open(pdf_document, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(reader.pages)
        
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text()

            if not text:
                images = convert_from_path(pdf_document, first_page=page_num + 1, last_page=page_num + 1)
                text = ''
                image = images[0]
                text += pytesseract.image_to_string(image, lang='rus')

            images = convert_from_path(pdf_document, first_page=page_num + 1, last_page=page_num + 1)
            image = images[0]
            result = yolo_model.predict(image, verbose=False)[0]

            height, width = result.orig_shape[0], result.orig_shape[1]

            annotator = Annotator(image, line_width=line_width, font_size=font_size)

            label_count = {"Formula": 0, "Picture": 0, "Table": 0}

            for label, box in zip(result.boxes.cls.tolist(), result.boxes.xyxyn.tolist()):
                label = int(label)  # Convert label to integer
                class_name = result.names[label]  # Get the class name for the label
                if class_name not in label_count:
                    continue
                x_min, y_min, x_max, y_max = int(box[0] * width), int(box[1] * height), int(box[2] * width), int(box[3] * height)
                img = np.array(image)
                cropped_img = img[y_min:y_max, x_min:x_max]

                label_count[class_name] += 1

                output_filename = f"{class_name}_{page_num + 1}page_{label_count[class_name]}.png"
                output_path = os.path.join(unstructured_output_dir, output_filename)

                cv2.imwrite(output_path, cropped_img)
                print(f'Saved: {output_filename}')

            text_output_file = os.path.join(text_output_dir, f"{file_name}_page_{page_num + 1}.txt")
            with open(text_output_file, "w", encoding="utf-8") as text_file:
                text_file.write(text)
#                 print(f"Text saved: {text_output_file}")

  0%|          | 0/17 [00:00<?, ?it/s]

Saved: Formula_43page_1.png
Saved: Table_61page_1.png
Saved: Table_62page_1.png
Saved: Table_62page_2.png
Saved: Picture_84page_1.png
Saved: Formula_85page_1.png
Saved: Picture_85page_1.png
Saved: Picture_87page_1.png
Saved: Formula_87page_1.png
Saved: Formula_88page_1.png
Saved: Formula_88page_2.png
Saved: Table_89page_1.png
Saved: Formula_89page_1.png
Saved: Table_90page_1.png
Saved: Table_91page_1.png
Saved: Table_92page_1.png
Saved: Table_93page_1.png
Saved: Table_94page_1.png
Saved: Table_95page_1.png
Saved: Table_96page_1.png
Saved: Picture_96page_1.png
Saved: Formula_97page_1.png
Saved: Formula_97page_2.png
Saved: Formula_99page_1.png
Saved: Table_100page_1.png
Saved: Formula_100page_1.png
Saved: Formula_100page_2.png
Saved: Table_101page_1.png
Saved: Table_101page_2.png
Saved: Table_102page_1.png
Saved: Picture_103page_1.png
Saved: Picture_104page_1.png
Saved: Formula_104page_1.png
Saved: Picture_104page_2.png
Saved: Formula_104page_2.png
Saved: Formula_104page_3.png
Saved: For

/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(


Saved: Table_3page_1.png


/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(


...

Saved: Table_96page_1.png
Saved: Table_96page_2.png


/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(


Saved: Table_97page_1.png


/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /SymbolSetEncoding not implemented yet
  warnings.warn(


Saved: Picture_1page_1.png
Saved: Table_3page_1.png
Saved: Table_6page_1.png
Saved: Table_7page_1.png
Saved: Formula_7page_1.png
Saved: Table_8page_1.png
Saved: Picture_9page_1.png
Saved: Table_10page_1.png
Saved: Table_11page_1.png
Saved: Table_12page_1.png
Saved: Table_12page_2.png
Saved: Picture_13page_1.png
Saved: Formula_14page_1.png
Saved: Formula_14page_2.png
Saved: Formula_15page_1.png
Saved: Formula_16page_1.png
Saved: Formula_16page_2.png
Saved: Formula_16page_3.png
Saved: Table_17page_1.png
Saved: Formula_17page_1.png
Saved: Table_18page_1.png
Saved: Table_18page_2.png
Saved: Formula_18page_1.png
Saved: Formula_19page_1.png
Saved: Formula_19page_2.png
Saved: Table_20page_1.png
Saved: Formula_20page_1.png
Saved: Table_21page_1.png
Saved: Formula_22page_1.png
Saved: Table_23page_1.png
Saved: Formula_23page_1.png
Saved: Formula_23page_2.png
Saved: Formula_23page_3.png
Saved: Formula_24page_1.png
Saved: Table_25page_1.png
Saved: Formula_25page_1.png
Saved: Formula_25page_2.png
S

In [22]:
import shutil
shutil.make_archive('data.zip', 'zip', '/kaggle/working/')

'/kaggle/working/data.zip.zip'